In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score

from sklearn.cluster import KMeans
import xgboost as xgb

In [2]:
x_train_path = '/Users/minki/prepare_dataq_test/dataset/X_train.csv'
x_test_path = '/Users/minki/prepare_dataq_test/dataset/X_test.csv'
y_train_path = '/Users/minki/prepare_dataq_test/dataset/y_train.csv'

x_train = pd.read_csv(x_train_path, encoding ='cp949')
x_test = pd.read_csv(x_test_path, encoding ='cp949')
y_train = pd.read_csv(y_train_path, encoding ='cp949')

In [3]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   cust_id  3500 non-null   int64  
 1   총구매액     3500 non-null   int64  
 2   최대구매액    3500 non-null   int64  
 3   환불금액     1205 non-null   float64
 4   주구매상품    3500 non-null   object 
 5   주구매지점    3500 non-null   object 
 6   내점일수     3500 non-null   int64  
 7   내점당구매건수  3500 non-null   float64
 8   주말방문비율   3500 non-null   float64
 9   구매주기     3500 non-null   int64  
dtypes: float64(3), int64(5), object(2)
memory usage: 273.6+ KB


In [4]:
# 먼저 cust_id는 int형이 아니라 Object형이다
x_train['cust_id'] = x_train['cust_id'].astype('object')
x_test['cust_id'] = x_test['cust_id'].astype('object')
y_train['cust_id'] = y_train['cust_id'].astype('object')

In [5]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   cust_id  3500 non-null   object 
 1   총구매액     3500 non-null   int64  
 2   최대구매액    3500 non-null   int64  
 3   환불금액     1205 non-null   float64
 4   주구매상품    3500 non-null   object 
 5   주구매지점    3500 non-null   object 
 6   내점일수     3500 non-null   int64  
 7   내점당구매건수  3500 non-null   float64
 8   주말방문비율   3500 non-null   float64
 9   구매주기     3500 non-null   int64  
dtypes: float64(3), int64(4), object(3)
memory usage: 273.6+ KB


In [6]:
# cust_id는 기본적으로 모델 구성에는 필요없는 데이터이므로 이를 드롭 및 삭제한다.
x_train_custId = x_train['cust_id']
x_test_custId = x_test['cust_id']

x_train = x_train.drop(['cust_id'], axis = 1)
x_test = x_test.drop(['cust_id'], axis = 1)

In [7]:
# 환불금액의 Null개수 조사
print(x_train.isnull().sum()['환불금액'])
print(x_test.isnull().sum()['환불금액'])

2295
1611


In [8]:
# 현재 환불금액의 Null개수가 매우 많으므로 결측값을 제거 대신에 채워줘야한다.
# 환불 금액이 Na인 것은 환불을 안했다는 것으로 0으로 채우겠다.

simpleImputer_train = SimpleImputer(strategy = 'constant')
simpleImputer_train.fit(pd.DataFrame(x_train['환불금액']))

x_train['환불금액'] = simpleImputer_train.transform(pd.DataFrame(x_train['환불금액']))


simpleImputer_test = SimpleImputer(strategy = 'constant')
simpleImputer_test.fit(pd.DataFrame(x_test['환불금액']))
x_test['환불금액'] = simpleImputer_test.transform(pd.DataFrame(x_test['환불금액']))

In [10]:
# 혹시 모르니 데이터를 복사해두자
x_train_copy = x_train.copy()
x_test_copy = x_test.copy()

In [21]:
# 주구매상품과 주구매지점이 String형태이므로 이를 object형태로 만들어주어야 한다.

x_train.loc[:, ['주구매상품', '주구매지점']] = x_train.loc[:, ['주구매상품', '주구매지점']].apply(LabelEncoder().fit_transform)
x_test.loc[:, ['주구매상품', '주구매지점']] = x_test.loc[:, ['주구매상품', '주구매지점']].apply(LabelEncoder().fit_transform)

In [22]:
# 여기서 주구매상품, 주구매지점을 제외하고 Scale을 변환해주어야 한다.
# MinMaxScaler를 사용해보자.

minMaxScaler_train = MinMaxScaler

,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,68282840,11264000,6860000.0,5,0,19,3.894737,0.527027,17
1,2136000,2136000,300000.0,21,19,2,1.500000,0.000000,1
2,3197000,1639000,0.0,6,1,2,2.000000,0.000000,1
3,16077620,4935000,0.0,5,2,18,2.444444,0.318182,16
4,29050000,24000000,0.0,15,8,2,1.500000,0.000000,85


In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

from sklearn.metrics import roc_auc_score

from sklearn.cluster import KMeans
import xgboost as xgb

x_train_path = 'data/X_train.csv'
x_test_path = 'data/X_test.csv'
y_train_path = 'data/y_train.csv'

x_train = pd.read_csv(x_train_path)
y_train = pd.read_csv(y_train_path)
x_test = pd.read_csv(x_test_path)

# cust_id의 형태 변환
x_train['cust_id'] = x_train['cust_id'].astype('object')
x_test['cust_id'] = x_test['cust_id'].astype('object')

# cust_id를 따로 저장 및 기존 데이터셋에서 분리

x_train_copy = x_train.copy()
x_test_copy = x_test.copy()

x_train_custID = x_train['cust_id']
x_test_custID = x_test['cust_id']

x_train = x_train.drop(['cust_id'], axis = 1)
x_test = x_test.drop(['cust_id'], axis = 1)

# 환불금액에서 null값이 보이므로 이를 0값으로 채움

# 환불금액의 null 값을 계산해보자

print(x_train.isnull().sum()['환불금액'])
print(x_test.isnull().sum()['환불금액'])

# null값을 0으로 채우자

simpleImputer_train = SimpleImputer(strategy = 'constant')
simpleImputer_test = SimpleImputer(strategy = 'constant')

simpleImputer_train.fit(pd.DataFrame(x_train['환불금액']))
simpleImputer_test.fit(pd.DataFrame(x_test['환불금액']))

x_train['환불금액'] = simpleImputer_train.transform(pd.DataFrame(x_train['환불금액']))
x_test['환불금액'] = simpleImputer_test.transform(pd.DataFrame(x_test['환불금액']))

# null값이 너무 많은 관계로 삭제할 수 없고 또 그렇다고 이 데이터를 그대로 이용하기에도 무리가 있다.
# 따라서 총구매액 - 환불금액 해서 최종구매액 행을 신설하자.

x_train['최종구매액'] = x_train['총구매액'] - x_train['환불금액'] 
x_test['최종구매액'] = x_test['총구매액'] - x_test['환불금액']

x_train['최종구매액'] = x_train['최종구매액'].apply(lambda x: x if(x > 0) else 0)
x_test['최종구매액'] = x_test['최종구매액'].apply(lambda x: x if(x > 0) else 0)

# 총구매액과 환불금액을 날리자

x_train = x_train.drop(['총구매액', '환불금액'], axis = 1)
x_test = x_test.drop(['총구매액', '환불금액'], axis = 1)

# 주구매상품과 주구매지점이 String 형식이므로 encoding이 필요하다.

x_train.loc[:, ['주구매상품', '주구매지점']] = x_train.loc[:, ['주구매상품', '주구매지점']].apply(LabelEncoder().fit_transform)
x_test.loc[:, ['주구매상품', '주구매지점']] = x_test.loc[:, ['주구매상품', '주구매지점']].apply(LabelEncoder().fit_transform)

# 이제 float형태 데이터들의 스케일을 맞춰주자.
# 스케일변환은 MinMaxScaler를 사용하도록 하겠다.

minMaxScaler_train = MinMaxScaler()
minMaxScaler_test = MinMaxScaler()

minMaxScaler_train.fit(x_train.iloc[:, 0, 3, 4, 5, 6, 7])
minMaxScaler_test.fit(x_test.iloc[:, 0, 3, 4, 5, 6, 7])

x_train.iloc[:, 0, 3, 4, 5, 6, 7] = minMaxScaler_train.transform(x_train.iloc[:, 0, 3, 4, 5, 6, 7])
x_test.iloc[:, 0, 3, 4, 5, 6, 7] = minMaxScaler_test.transform(x_test.iloc[:, 0, 3, 4, 5, 6, 7])

print(x_train.head())
print(x_test.head())













